#### Problem Statement / Objective

You need to analyze historical trade data from various Binance accounts over 90 days and evaluate their performance based on multiple financial metrics. The goal is to rank the accounts based on these metrics and identify the top 20 best-performing accounts.

Key Tasks:

1. Data Cleaning & Exploration:

- Load and inspect the dataset.
- Handle missing or inconsistent values.
- Understand the trade structure (e.g., Port_ID, timestamp, side, price, quantity, etc.).

2. Feature Engineering & Metrics Calculation:

Calculate financial performance metrics for each account, including:

- ROI (Return on Investment)
- PnL (Profit and Loss)
- Sharpe Ratio (Risk-adjusted returns)
- MDD (Maximum Drawdown) (Biggest loss from peak)
- Win Rate (% of profitable trades)
- Win Positions (Number of profitable trades)
- Total Positions (Total number of trades)

3. Ranking Algorithm:

- Develop a scoring system based on the calculated metrics.
- Assign weights to different metrics to determine overall ranking.
- Identify the top 20 accounts based on performance.

4. Documentation & Report:

- Explain methodology, assumptions, and findings in a structured report.
- Provide a final ranked list of the top 20 accounts.


In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 1. Load and Preprocess Data


In [47]:
# Assuming the data is in a CSV with Port_ID and Trade_History columns
df = pd.read_csv("D:\\Assignment\\Assinment\\TRADES_CopyTr_90D_ROI.csv")

In [48]:
import ast


# Function to safely parse JSON-like strings
def safe_json_loads(x):
    try:
        if pd.isna(x) or x.strip() == "":
            return None  # Handle empty values
        return eval(x)  # Convert string to a Python list safely
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None  # Handle errors

In [49]:
# Parse JSON column
df["Trade_History"] = df["Trade_History"].apply(safe_json_loads)

In [50]:
# Drop rows with missing or invalid Trade_History
df = df.dropna(subset=["Trade_History"])

In [51]:
df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


In [52]:
# Explode the JSON list into individual rows
trade_df = df.explode("Trade_History").reset_index(drop=True)

In [53]:
trade_df.head()

,Port_IDs,Trade_History
0,3925368433214965504,"{'time': 1718899656000, 'symbol': 'SOLUSDT', '..."
1,3925368433214965504,"{'time': 1718899618000, 'symbol': 'DOGEUSDT', ..."
2,3925368433214965504,"{'time': 1718899618000, 'symbol': 'DOGEUSDT', ..."
3,3925368433214965504,"{'time': 1718899616000, 'symbol': 'DOGEUSDT', ..."
4,3925368433214965504,"{'time': 1718899616000, 'symbol': 'DOGEUSDT', ..."


In [54]:
# Convert JSON dictionaries to columns
trade_df = pd.concat(
    [
        trade_df.drop(["Trade_History"], axis=1),
        trade_df["Trade_History"].apply(pd.Series),
    ],
    axis=1,
)

In [55]:
trade_df.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy
0,3925368433214965504,1718899656000,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True
1,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False
2,3925368433214965504,1718899618000,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False
3,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True
4,3925368433214965504,1718899616000,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True


#### 2. Data Cleaning


In [56]:
trade_df.isnull().sum()

Port_IDs               0
time                   0
symbol                 0
side                   0
price                  0
fee                    0
feeAsset               0
quantity               0
quantityAsset          0
realizedProfit         0
realizedProfitAsset    0
baseAsset              0
qty                    0
positionSide           0
activeBuy              0
dtype: int64

In [57]:
# Convert timestamp
trade_df["time"] = pd.to_datetime(trade_df["time"])

#### 3. Position Classification


In [58]:
conditions = [
    (trade_df["side"] == "BUY") & (trade_df["positionSide"] == "LONG"),
    (trade_df["side"] == "SELL") & (trade_df["positionSide"] == "LONG"),
    (trade_df["side"] == "SELL") & (trade_df["positionSide"] == "SHORT"),
    (trade_df["side"] == "BUY") & (trade_df["positionSide"] == "SHORT"),
]
choices = ["long_open", "long_close", "short_open", "short_close"]
trade_df["position_type"] = np.select(conditions, choices, default=None)

In [59]:
# Filter valid positions
trade_df = trade_df[trade_df["position_type"].notna()]

In [60]:
trade_df.columns

Index(['Port_IDs', 'time', 'symbol', 'side', 'price', 'fee', 'feeAsset',
       'quantity', 'quantityAsset', 'realizedProfit', 'realizedProfitAsset',
       'baseAsset', 'qty', 'positionSide', 'activeBuy', 'position_type'],
      dtype='object')

In [61]:
trade_df.head()

,Port_IDs,time,symbol,side,price,fee,feeAsset,quantity,quantityAsset,realizedProfit,realizedProfitAsset,baseAsset,qty,positionSide,activeBuy,position_type
0,3925368433214965504,1970-01-01 00:28:38.899656,SOLUSDT,BUY,132.53700,-0.994027,USDT,1988.05500,USDT,0.0,USDT,SOL,15.0,LONG,True,long_open
1,3925368433214965504,1970-01-01 00:28:38.899618,DOGEUSDT,BUY,0.12182,-0.279796,USDT,1398.98088,USDT,0.0,USDT,DOGE,11484.0,LONG,False,long_open
2,3925368433214965504,1970-01-01 00:28:38.899618,DOGEUSDT,BUY,0.12182,-0.039494,USDT,197.47022,USDT,0.0,USDT,DOGE,1621.0,LONG,False,long_open
3,3925368433214965504,1970-01-01 00:28:38.899616,DOGEUSDT,BUY,0.12182,-0.008284,USDT,16.56752,USDT,0.0,USDT,DOGE,136.0,LONG,True,long_open
4,3925368433214965504,1970-01-01 00:28:38.899616,DOGEUSDT,BUY,0.12182,-0.046109,USDT,92.21774,USDT,0.0,USDT,DOGE,757.0,LONG,True,long_open


#### 4. Metric Calculation


In [62]:
from collections import defaultdict, deque

In [63]:
def calculate_metrics(group):
    queues = defaultdict(deque)
    realized_profits = []
    open_quantities = []
    cumulative_pnl = []
    current_cumulative = 0

    for _, trade in group.sort_values("time").iterrows():
        pos_type = trade["position_type"]
        asset = trade["baseAsset"]

        if "_open" in pos_type:
            # Add to queue
            key = (asset, pos_type.split("_")[0])
            queues[key].append({"quantity": trade["quantity"], "time": trade["time"]})
        else:
            # Match with earliest open position
            key = (asset, pos_type.split("_")[0])
            if queues[key]:
                open_trade = queues[key].popleft()
                realized_profits.append(trade["realizedProfit"])
                open_quantities.append(open_trade["quantity"])

                # Track cumulative PnL for MDD
                current_cumulative += trade["realizedProfit"]
                cumulative_pnl.append(current_cumulative)

    # Calculate metrics
    total_pos = len(realized_profits)
    win_pos = sum(p > 0 for p in realized_profits)

    return pd.Series(
        {
            "ROI": (
                (sum(realized_profits) / sum(open_quantities) * 100)
                if open_quantities
                else 0
            ),
            "PnL": sum(realized_profits),
            "Sharpe Ratio": (
                (np.mean(realized_profits) / np.std(realized_profits))
                if len(realized_profits) > 1
                else 0
            ),
            "MDD": calculate_mdd(cumulative_pnl),
            "Win Rate": win_pos / total_pos if total_pos else 0,
            "Win Positions": win_pos,
            "Total Positions": total_pos,
        }
    )

In [64]:
def calculate_mdd(cumulative_pnl):
    if not cumulative_pnl:
        return 0
    running_max = np.maximum.accumulate(cumulative_pnl)
    drawdowns = running_max - cumulative_pnl
    return np.max(drawdowns) if len(drawdowns) > 0 else 0

In [65]:
# Calculate metrics per Port_ID
metrics_df = trade_df.groupby("Port_IDs").apply(calculate_metrics).reset_index()

C:\Users\ANAS KHAN\AppData\Local\Temp\ipykernel_12564\693745166.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics_df = trade_df.groupby("Port_IDs").apply(calculate_metrics).reset_index()


#### 5. Ranking System


In [66]:
# Normalization
metrics = ["ROI", "PnL", "Sharpe Ratio", "MDD", "Win Rate"]
weights = {"ROI": 0.25, "PnL": 0.25, "Sharpe Ratio": 0.2, "MDD": 0.15, "Win Rate": 0.15}

for metric in metrics:
    if metric != "MDD":
        metrics_df[f"{metric}_norm"] = (
            metrics_df[metric] - metrics_df[metric].min()
        ) / (metrics_df[metric].max() - metrics_df[metric].min())
    else:
        # Invert MDD since lower is better
        metrics_df[f"{metric}_norm"] = 1 - (
            (metrics_df[metric] - metrics_df[metric].min())
            / (metrics_df[metric].max() - metrics_df[metric].min())
        )

In [67]:
# Calculate composite score
metrics_df["Composite_Score"] = sum(
    metrics_df[f"{metric}_norm"] * weights[metric] for metric in weights
)

# 6. Final Output


In [68]:
# Get top 20 accounts
top_20 = metrics_df.sort_values("Composite_Score", ascending=False).head(20)

# Save results
metrics_df.to_csv("all_account_metrics.csv", index=False)
top_20[["Port_IDs", "Composite_Score"]].to_csv("top_20_accounts.csv", index=False)